# Tracklock

scrape deadlock data

convert into DataFrame

export as .csv

Example url, playerID for "Ediorian": https://tracklock.gg/players/1676299122/matches?page=1

In [2]:
import requests # to scrape the data
from bs4 import BeautifulSoup # to handle the data
import re # for getting wins AND losses 
import pandas as pd # for dataframes
import csv # to create csv file 
import os # to check if file exsists already

### Pull the data:

In [3]:
# get tracklock page respons with requests
# pull data of "player" (current "Ediorian")
playerID = 1676299122
res = requests.get(f"https://tracklock.gg/players/{playerID}/matches?page=1")
res

<Response [200]>

### Read the data with BeautifulSoup

This is how to extract different data.

later full function with each part.

In [4]:
soup = BeautifulSoup(res.text, 'html.parser')
#soup

In [5]:
games = soup.findAll("tr", class_="transition-colors")

C:\Users\Nikla_000\AppData\Local\Temp\ipykernel_20388\2959419742.py:1: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  games = soup.findAll("tr", class_="transition-colors")


In [15]:
games[-1]

<tr class="transition-colors data-[state=selected]:bg-muted even:bg-[#0F1116] odd:bg-[#11141b] border-b border-[#363636f7] hover:bg-gray-750 text-white"><td class="p-2 align-middle [&amp;:has([role=checkbox])]:pr-0 [&amp;&gt;[role=checkbox]]:translate-y-[2px]"><div class="flex items-center gap-2"><img alt="The Magnificent Sinclair" class="w-12 h-12 rounded" data-nimg="1" decoding="async" height="48" loading="lazy" src="/_next/image?url=%2Fpanorama%2Fimages%2Fheroes%2Fmagician_sm_psd.png&amp;w=96&amp;q=75" srcset="/_next/image?url=%2Fpanorama%2Fimages%2Fheroes%2Fmagician_sm_psd.png&amp;w=48&amp;q=75 1x, /_next/image?url=%2Fpanorama%2Fimages%2Fheroes%2Fmagician_sm_psd.png&amp;w=96&amp;q=75 2x" style="color:transparent" width="48"/><div><span class="font-medium">The Magnificent Sinclair</span><div class="text-sm text-gray-400">32850808</div></div></div></td><td class="p-2 align-middle [&amp;:has([role=checkbox])]:pr-0 [&amp;&gt;[role=checkbox]]:translate-y-[2px]"><div class="flex flex-col

### Find the wins and losses

**Losses:**
tag = span, class "text-red-500 font-medium"

**Wins:**
tag = span, class "text-green-500 font-medium"


In [16]:
# re.compile lets us get both wins and losses, with | -> "or"
winLoss = soup.findAll("span", class_=re.compile(r"text-red-500 font-medium|text-green-500 font-medium")) # works like OR now 
len(winLoss)

20

In [17]:
winLoss[0]

<span class="text-green-500 font-medium">Win</span>

**save the text in a list:**

In [18]:
winLossData = []
for game in winLoss:
    gameStatus = game.text
    winLossData.append(gameStatus)
winLossData

['Win',
 'Win',
 'Win',
 'Loss',
 'Loss',
 'Win',
 'Win',
 'Loss',
 'Loss',
 'Loss',
 'Loss',
 'Win',
 'Loss',
 'Loss',
 'Loss',
 'Loss',
 'Win',
 'Loss',
 'Loss',
 'Win']

### Dates: 


**Dates:**
tag = span, class "text-sm text-gray-400"

In [19]:
dates = soup.findAll("span", class_="text-sm text-gray-400")
dates[0]

<span class="text-sm text-gray-400">Feb 15, 2025</span>

**save the dates in a list:**

In [20]:
dateData = []
for game in dates:
    date = game.text
    dateData.append(date)
dateData[:3]

['Feb 15, 2025', 'Feb 15, 2025', 'Feb 15, 2025']

### Duration: 

**Durations:**
tag = div, class = flex flex-col gap-1

this gets us both the match length and date so we do "every other".

In [21]:
durations = soup.findAll("div", class_="flex flex-col gap-1")
durations[0]

<div class="flex flex-col gap-1"><span>24<!-- --> min</span><svg height="6" width="120"><rect fill="transparent" height="6" rx="3" ry="3" width="120"></rect><rect fill="rgb(145, 162, 189)" height="6" rx="3" ry="3" width="65.53159851301115"></rect></svg></div>

**save the match lengths in a list:**

In [22]:
durationData = []
for i, game in enumerate(durations):
    if (i % 2 == 0): # every other [VERY SKETCHY FIX]
        duration = game.text
        durationData.append(duration)
durationData[:5]

['24 min', '28 min', '33 min', '32 min', '28 min']

### Convert to DataFrame

In [39]:
# traditional loop
rows = [["Win/Loss", "Match length", "Date played"]]# define column names

for i, game in enumerate(winLossData):
    row = [winLossData[i], durationData[i], dateData[i]]
    rows.append(row)
#rows

In [24]:
# create the pandas dataframe 
df = pd.DataFrame(rows)
df.head(3)

,0,1,2
0,Win/Loss,Match length,Date played
1,Win,24 min,"Feb 15, 2025"
2,Win,28 min,"Feb 15, 2025"


In [43]:
# make it prettier
df.columns = df.iloc[0] # first row as column names
df = df.drop(0).reset_index(drop=True) # remove the row and reset index
df

,Win,"Feb 15, 2025",28 min
0,Win,"Feb 15, 2025",33 min
1,Loss,"Feb 14, 2025",32 min
2,Loss,"Feb 14, 2025",28 min
3,Win,"Feb 14, 2025",30 min
4,Win,"Feb 14, 2025",26 min
...,...,...,...
1574,Loss,"Jun 20, 2024",22 min
1575,Win,"Jun 20, 2024",32 min
1576,Loss,"Jun 20, 2024",38 min
1577,Loss,"Jun 20, 2024",30 min


In [26]:
# describe, shape, type
print("describe:\n", df.describe(), end="\n\n")
print("shape:\n", df.shape, end="\n\n")
print("type:\n", type(df))

describe:
 0      Win/Loss Match length   Date played
count        20           20            20
unique        2           13             3
top        Loss       28 min  Feb 12, 2025
freq         12            3            13

shape:
 (20, 3)

type:
 <class 'pandas.core.frame.DataFrame'>


In [27]:
print(df["Match length"].max())
print(df["Match length"].min())

42 min
20 min


### as Functions

- in order to get many pages
- easily control the pulled data

In [29]:
# pull the data of all available pages and return them as a list of pages
def pullTracklockPages(playerId):
    """pull all available data for given player. Return list of all pages."""
    # get tracklock page respons with requests 
    res = requests.get(f"https://tracklock.gg/players/{playerId}/matches?page=1")
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # get total pages number
    pageNumbers = soup.findAll("li")
    totalPageNumber = int(pageNumbers[-2].text)
    print("total pages to import:", totalPageNumber)
    
    # get all pages
    allPages = []
    for pageNumber in range(1, totalPageNumber + 1):
        res = requests.get(f"https://tracklock.gg/players/{playerId}/matches?page={pageNumber}")
        
        print("Page", pageNumber, "was pulled.")
        
        soup = BeautifulSoup(res.text, 'html.parser')
        allPages.append(soup)
    return allPages

In [30]:
# pull data for "Edorian" and save as variable
allPagesData = pullTracklockPages(playerID) 

total pages to import: 80
Page 1 was pulled.
Page 2 was pulled.
Page 3 was pulled.
Page 4 was pulled.
Page 5 was pulled.
Page 6 was pulled.
Page 7 was pulled.
Page 8 was pulled.
Page 9 was pulled.
Page 10 was pulled.
Page 11 was pulled.
Page 12 was pulled.
Page 13 was pulled.
Page 14 was pulled.
Page 15 was pulled.
Page 16 was pulled.
Page 17 was pulled.
Page 18 was pulled.
Page 19 was pulled.
Page 20 was pulled.
Page 21 was pulled.
Page 22 was pulled.
Page 23 was pulled.
Page 24 was pulled.
Page 25 was pulled.
Page 26 was pulled.
Page 27 was pulled.
Page 28 was pulled.
Page 29 was pulled.
Page 30 was pulled.
Page 31 was pulled.
Page 32 was pulled.
Page 33 was pulled.
Page 34 was pulled.
Page 35 was pulled.
Page 36 was pulled.
Page 37 was pulled.
Page 38 was pulled.
Page 39 was pulled.
Page 40 was pulled.
Page 41 was pulled.
Page 42 was pulled.
Page 43 was pulled.
Page 44 was pulled.
Page 45 was pulled.
Page 46 was pulled.
Page 47 was pulled.
Page 48 was pulled.
Page 49 was pulled.
Pag

In [31]:
len(allPagesData)

80

In [32]:
# function for getting the data from the pages
def scrapeTracklock(pages):
    """Scrape the data of of the pages that are given. Return """

    winLossData = []
    dateData = []
    durationData = []

    for page in pages:
        # Extract Win/Loss data
        winLoss = page.findAll("span", class_=re.compile(r"text-red-500 font-medium|text-green-500 font-medium"))
        winLossData.extend(game.text for game in winLoss)

        # Extract Dates
        dates = page.findAll("span", class_="text-sm text-gray-400")
        dateData.extend(date.text for date in dates)

        # Extract Match Durations
        durations = page.findAll("div", class_="flex flex-col gap-1")
        for i, game in enumerate(durations):
            if i % 2 == 0:  # Every other one
                durationData.append(game.text)

    
    return winLossData, dateData, durationData

In [ ]:
# use scrapeTacklock() on pulled pages
fullData = scrapeTracklock(allPagesData)

(['Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss

In [41]:
fullData = scrapeTracklock(allPagesData)
#fullData

### Export data as csv

first convert into pandas dataframe

In [ ]:
def exportData(data, playerID):
    # data into a pd.DataFrame
    df = pd.DataFrame({
        'Result': data[0],
        'Date': data[1],
        'Duration': data[2]
    })

    # create CSV file out of dataframe
    if (not os.path.exists(f'playerID{playerID}_dl_stats.csv')): #create new csv
        df.to_csv(f'playerID{playerID}_dl_stats.csv', index=False)
        print("new csv file created.")
    else: 
        print("File already exists.")

  Result          Date Duration
0    Win  Feb 15, 2025   24 min
1    Win  Feb 15, 2025   28 min
2    Win  Feb 15, 2025   33 min
3   Loss  Feb 14, 2025   32 min
4   Loss  Feb 14, 2025   28 min


In [ ]:
exportData(fullData)

### CONTINUE:

add to scrapeTracklock() more data pulle